In [1]:
import pandas as pd

In [24]:
neer=pd.read_csv('neerja.csv')
neer.columns
neer=neer[['Key','Defect Type', 'Root Cause', 'Injection Point', 'Detection Point','Notes']]
neer.columns

Index(['Key', 'Defect Type', 'Root Cause', 'Injection Point',
       'Detection Point', 'Notes'],
      dtype='object')

In [20]:
neer['Notes'].unique()
# df[df['A'].isna()]

array([nan, 'Not a defect. Placeholder.',
       'Not a defect.Placeholder for UAT defects.', 'Not a defect.',
       'Not a defect. Listed as placeholder.'], dtype=object)

In [23]:
df=pd.read_csv('all_old.csv')
df.columns

Index(['Key', 'Parent Key', 'Sprint Name', 'Sprint Start Date',
       'Sprint End Date', 'Cycle', 'Project', 'Team', 'Product', 'Org_Id',
       'Created', 'Resolved', 'Year', 'Number of days', 'Status',
       'Defect Type', 'Root Cause', 'Injection Point', 'Detection Point',
       'Detection Group'],
      dtype='object')

In [8]:
df.iloc[0]

Key                                     USAS-1749
Parent Key                                    NaN
Sprint Name          USAS Implementation Sprint 8
Sprint Start Date                      2024-06-25
Sprint End Date                        2024-07-15
Cycle                                     2024-Q2
Project                                      USAS
Team                                          NaN
Product                                       NaN
Org_Id                                    faausas
Created                                06-27-2024
Resolved                                      NaN
Year                                         2024
Number of days                                NaN
Status                                      To Do
Defect Type          Logical Bugs/Functional Bugs
Root Cause                                    NaN
Injection Point                               NaN
Detection Point                    In Development
Detection Group                       Development


In [25]:
import numpy as np
df['Notes']=np.nan
columns_to_update = ['Detection Point', 'Defect Type', 'Root Cause', 'Injection Point', 'Notes']


for column in columns_to_update:
    issue_key_to_value = pd.Series(neer[column].values, index=neer['Key']).to_dict()
    df[column] = df[column].fillna(df['Key'].map(issue_key_to_value))

df= df[df['Notes'].isna()]

In [26]:
df.to_csv('defects.csv',index=False)

In [27]:
sprint_data_df = pd.read_csv('sprint.csv')
defects_df = pd.read_csv('defects.csv')

In [28]:
sprint_data_df.insert(sprint_data_df.columns.get_loc('uat_defects') + 1, 'other_defects', 0)
sprint_data_df['other_defects']

0    0
1    0
2    0
3    0
4    0
5    0
6    0
7    0
Name: other_defects, dtype: int64

In [29]:
defects_df['Detection Group'].unique()

array(['Development', 'Testing', 'Review', 'UAT'], dtype=object)

In [30]:
for idx, row in sprint_data_df.iterrows():
    sprint_name = row['sprint_name']
    
    # Filter defects for the current sprint
    sprint_defects = defects_df[defects_df['Sprint Name'] == sprint_name]
    
    # Count defects by detection group and update the columns
    sprint_data_df.at[idx, 'defects_count'] = len(sprint_defects)
    sprint_data_df.at[idx, 'development_defects'] = (sprint_defects['Detection Group'] == 'Development').sum()
    sprint_data_df.at[idx, 'review_defects'] = (sprint_defects['Detection Group'] == 'Review').sum()
    sprint_data_df.at[idx, 'testing_defects'] = (sprint_defects['Detection Group'] == 'Testing').sum()
    
    sprint_data_df.at[idx, 'uat_defects'] = (sprint_defects['Detection Group'] == 'UAT').sum()
    sprint_data_df.at[idx, 'other_defects'] = sprint_defects[
        ~sprint_defects['Detection Group'].isin(['Development', 'Review', 'UAT','Testing'])
    ].shape[0]

# Save the updated sprint_data to a new Excel file
output_file = 'updated_sprint_data.csv'
sprint_data_df.to_csv(output_file, index=False)

print(f"Updated sprint data saved to {output_file}")


Updated sprint data saved to updated_sprint_data.csv


In [31]:
# Load the updated sprint_data
sprint_data_df = pd.read_csv('updated_sprint_data.csv')

# Calculate the new columns based on the conditions provided
sprint_data_df['defect_density_points'] = sprint_data_df.apply(
    lambda row: row['defects_count'] / row['story_points_completed'] if row['story_points_completed'] else 0, axis=1
)

sprint_data_df['weighted_defect_density_points'] = sprint_data_df.apply(
    lambda row: (row['review_defects'] + row['development_defects'] + 
                 2 * row['testing_defects'] + 3 * row['uat_defects'] + 
                 5 * row.get('production_defects', 0)) / row['story_points_completed']
    if row['story_points_completed'] else 0, axis=1
)

sprint_data_df['development_defect_density'] = sprint_data_df.apply(
    lambda row: row['development_defects'] / row['story_points_completed'] if row['story_points_completed'] else 0, axis=1
)

sprint_data_df['review_defect_density'] = sprint_data_df.apply(
    lambda row: row['review_defects'] / row['story_points_completed'] if row['story_points_completed'] else 0, axis=1
)

sprint_data_df['testing_defect_density'] = sprint_data_df.apply(
    lambda row: row['testing_defects'] / row['story_points_completed'] if row['story_points_completed'] else 0, axis=1
)

sprint_data_df['development_and_review_defect_density'] = sprint_data_df.apply(
    lambda row: (row['development_defects'] + row['review_defects']) / row['story_points_completed']
    if row['story_points_completed'] else 0, axis=1
)

# Save the updated DataFrame to a new Excel file
output_file = 'updated_sprint_data_with_density.csv'
sprint_data_df.to_csv(output_file, index=False)

print(f"Data with defect density metrics saved to {output_file}")

Data with defect density metrics saved to updated_sprint_data_with_density.csv
